In [1]:
import os
import sys
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from model_settings import ms
from df_collector import df_collector
root = Path().resolve().parent.parent
df_collector.root = root
options = df_collector.cboe_spx_barriers()
options

,Unnamed: 0,spot_price,strike_price,barrier,days_to_maturity,updown,outin,w,barrier_type_name,rebate,dividend_rate,risk_free_rate,theta,kappa,rho,eta,v0,calculation_date,date,barrier_price
0,0.0,4321.5,3889.35,2160.75,60,Down,Out,call,DownOut,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,480.947376
1,1.0,4321.5,3889.35,2160.75,60,Down,Out,put,DownOut,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,20.926593
2,2.0,4321.5,3889.35,2160.75,60,Down,In,call,DownIn,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,0.000000
3,3.0,4321.5,3889.35,2160.75,60,Down,In,put,DownIn,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,1.205822
4,4.0,4321.5,3889.35,2160.75,90,Down,Out,call,DownOut,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,504.346856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31021915,NaN,5813.5,6394.85,8720.25,540,Up,In,put,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,6.577341
31021916,NaN,5813.5,6394.85,8720.25,720,Up,Out,call,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,377.391470
31021917,NaN,5813.5,6394.85,8720.25,720,Up,Out,put,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,650.459798
31021918,NaN,5813.5,6394.85,8720.25,720,Up,In,call,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,189.459253


In [2]:
dataset = options.copy().iloc[:,1:].dropna().reset_index(drop=True)
dataset

,spot_price,strike_price,barrier,days_to_maturity,updown,outin,w,barrier_type_name,rebate,dividend_rate,risk_free_rate,theta,kappa,rho,eta,v0,calculation_date,date,barrier_price
0,4321.5,3889.35,2160.75,60,Down,Out,call,DownOut,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,480.947376
1,4321.5,3889.35,2160.75,60,Down,Out,put,DownOut,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,20.926593
2,4321.5,3889.35,2160.75,60,Down,In,call,DownIn,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,0.000000
3,4321.5,3889.35,2160.75,60,Down,In,put,DownIn,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,1.205822
4,4321.5,3889.35,2160.75,90,Down,Out,call,DownOut,0.0,0.0,0.04,0.035128,17.741791,-0.543376,3.563427,0.032496,2021-10-01 11:34:32.952,2021-10-01,504.346856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30897130,5813.5,6394.85,8720.25,540,Up,In,put,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,6.577341
30897131,5813.5,6394.85,8720.25,720,Up,Out,call,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,377.391470
30897132,5813.5,6394.85,8720.25,720,Up,Out,put,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,650.459798
30897133,5813.5,6394.85,8720.25,720,Up,In,call,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-15,189.459253


In [3]:
price_name = 'barrier_price' 
dataset = dataset[dataset['barrier_price']<dataset['spot_price']].dropna().reset_index(drop=True).copy()

In [6]:
dataset['observed_price'] = np.maximum(dataset[price_name] + np.random.normal(scale=(0.15)**2,size=dataset.shape[0]),0)
dataset['relative_moneyness'] = dataset['spot_price']/dataset['strike_price']
dataset['relative_barrier'] = dataset['barrier']/dataset['strike_price']
dataset['relative_price'] = dataset['observed_price']/dataset['strike_price']
dates = dataset['date'].drop_duplicates().copy().reset_index(drop=True)
dates
# dataset = dataset[(dataset['w']=='put')&(dataset['barrier_type_name']=='DownOut')].reset_index(drop=True)

0      2021-10-01
1      2021-10-04
2      2021-10-05
3      2021-10-06
4      2021-10-07
          ...    
757    2024-10-09
758    2024-10-10
759    2024-10-11
760    2024-10-14
761    2024-10-15
Name: date, Length: 762, dtype: object

In [8]:
t = pd.Series(np.sort(dataset['date'].unique()))
development_dates = dates[:len(dates)//4]
test_dates = dates[~dates.isin(development_dates)]
train_data = dataset[dataset['date'].isin(development_dates)]
test_data = dataset[dataset['date'].isin(development_dates)]


In [11]:
from convsklearn import convsklearn
trainer = convsklearn()
trainer.excluded_features = trainer.excluded_features + ['spot_price','strike_price','barrier']
trainer.excluded_features

['barrier_price',
 'asian',
 'observed_price',
 'outin',
 'updown',
 'n_fixings',
 'spot_price',
 'strike_price',
 'barrier']